In [82]:
import os
from dataclasses import dataclass
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from config import (
    countries,
    dap_bidding_zones,
    interconnections,
    interconnections_edge_matrix,
)
from tqdm import tqdm
from dotenv import load_dotenv

import math
import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch_geometric.nn import GATv2Conv
from torch_geometric.data import Data
from torch.nn import BatchNorm1d

In [83]:
load_dotenv()
engine = create_engine(os.getenv("SQLALCHEMY_DATABASE_URI"))

In [84]:
flow_df = pd.read_sql_table("flow_32", engine)
flow_df = flow_df.set_index("DateTime")
flow_df.fillna(0, inplace=True)

In [85]:
dap_df = pd.DataFrame()
for country_id in countries.keys():
    dap_df[country_id] = pd.read_sql_table(f"{country_id}_dap", engine).set_index(
        "DateTime"
    )
dap_df.index = pd.to_datetime(dap_df.index)
dap_df.ffill(inplace=True)
dap_df.fillna(0, inplace=True)

In [86]:
datetime_intersect = flow_df.index.intersection(dap_df.index)
print(len(datetime_intersect))
print(min(datetime_intersect), max(datetime_intersect))
# Check if datetime_intersect is monotonically increasing
assert all(
    datetime_intersect[i] < datetime_intersect[i + 1]
    for i in range(len(datetime_intersect) - 1)
)

43729
2015-01-04 23:00:00 2019-12-31 23:00:00


In [87]:
flow_df = flow_df.loc[datetime_intersect]
dap_df = dap_df.loc[datetime_intersect]

In [88]:
edges = np.array(interconnections_edge_matrix)
print(edges.shape)
# Map edge names to indices
edge_names = np.unique(edges)
edge_map = {edge: i for i, edge in enumerate(edge_names)}
edge_indices = np.array([edge_map[edge] for edge in edges.flatten()]).reshape(
    edges.shape
)
# Repeat edge indices for each datetime
edge_indices = np.repeat(
    edge_indices[np.newaxis, :, :],
    len(datetime_intersect),
    axis=0,
)
print(edge_indices.shape)
n_edges = edges.shape[1]

(2, 32)
(43729, 2, 32)


In [89]:
# Edge labels (flow) of shape (n_datetime, n_edges, 1)
edge_labels = np.array(flow_df)
# print(edge_labels.shape)
edge_labels = np.reshape(
    edge_labels, (len(datetime_intersect), edge_labels.shape[1], 1)
)
print(edge_labels.shape)
edge_attributes = np.copy(edge_labels)
print(edge_attributes.shape)

(43729, 32, 1)
(43729, 32, 1)


In [90]:
# Node features (dap)
node_features = np.array(dap_df)
# print(node_features.shape)
node_features = np.reshape(
    node_features, (len(datetime_intersect), node_features.shape[1], 1)
)
print(node_features.shape)
n_nodes = node_features.shape[1]

(43729, 10, 1)


In [91]:
assert (
    len(datetime_intersect)
    == edge_indices.shape[0]
    == edge_labels.shape[0]
    == edge_attributes.shape[0]
    == node_features.shape[0]
)

In [92]:
# Print a snapshot of the graph data
idx = 0
print(datetime_intersect[idx])
print(edge_indices[idx])
print(edge_labels[idx])
print(edge_attributes[idx])
print(node_features[idx])

2015-01-04 23:00:00
[[0 0 0 0 3 3 3 4 4 4 4 2 2 2 2 2 5 6 6 7 7 7 7 7 8 8 1 1 9 9 9 9]
 [4 6 7 9 2 7 8 0 2 1 9 3 4 6 7 1 9 0 2 0 3 2 8 9 3 7 4 2 0 4 5 7]]
[[   0.  ]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [1315.79]
 [  52.  ]
 [ 617.  ]
 [ 279.  ]
 [1433.  ]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [3205.  ]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [2106.86]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [ 964.  ]
 [   0.  ]
 [ 704.  ]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [ 169.19]
 [   0.  ]]
[[   0.  ]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [1315.79]
 [  52.  ]
 [ 617.  ]
 [ 279.  ]
 [1433.  ]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [3205.  ]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [2106.86]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [ 964.  ]
 [   0.  ]
 [ 704.  ]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [ 169.19]
 [   0.  ]]
[[36.56 ]
 [28.625]
 [36.56 ]
 [22.34 ]
 [49.14 ]
 [22.34 ]
 [36.56 ]
 [28.538]
 [37.8  ]
 [42.86 ]]


In [93]:
# https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.conv.GATv2Conv.html
class GNNEncoder(nn.Module):
    def __init__(
        self, hidden_channels, num_heads_GAT, dropout_p_GAT, edge_dim_GAT, momentum_GAT
    ):
        super().__init__()
        self.gat = GATv2Conv(
            (-1, -1),
            hidden_channels,
            add_self_loops=False,
            heads=num_heads_GAT,
            edge_dim=edge_dim_GAT,
        )
        self.norm = BatchNorm1d(
            hidden_channels,
            momentum=momentum_GAT,
            affine=False,
            track_running_stats=False,
        )
        self.dropout = nn.Dropout(dropout_p_GAT)

    def forward(self, x, edge_indices, edge_attrs):
        x = self.dropout(x)
        x = self.norm(x)
        nodes_embedds = self.gat(x, edge_indices, edge_attrs)
        nodes_embedds = F.leaky_relu(nodes_embedds, negative_slope=0.1)
        return nodes_embedds

In [94]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model)
        )
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer("pe", pe)

    def forward(self, x: Tensor) -> Tensor:
        x = x + self.pe[: x.size(0)]
        return self.dropout(x)

In [95]:
class Transformer(nn.Module):
    def __init__(
        self,
        dim_model,
        num_heads_TR,
        num_encoder_layers_TR,
        num_decoder_layers_TR,
        dropout_p_TR,
    ):
        super().__init__()
        self.pos_encoder = PositionalEncoding(dim_model)
        self.transformer = nn.Transformer(
            d_model=dim_model,
            nhead=num_heads_TR,
            num_decoder_layers=num_encoder_layers_TR,
            num_encoder_layers=num_decoder_layers_TR,
            dropout=dropout_p_TR,
        )

    def forward(self, src, trg):
        src = self.pos_encoder(src)
        trg = self.pos_encoder(trg)
        temporal_node_embeddings = self.transformer(src, trg)
        return temporal_node_embeddings

In [96]:
class EdgeDecoder(nn.Module):
    def __init__(self, hidden_channels, num_heads_GAT, num_edges, num_nodes):
        super().__init__()
        self.lin1 = nn.Linear(
            num_nodes * hidden_channels * num_heads_GAT, hidden_channels
        )
        self.lin2 = nn.Linear(hidden_channels, num_edges)

    def forward(self, x):
        # Flatten the tensor
        x = torch.flatten(x)
        x = self.lin1(x)
        x = F.leaky_relu(x, negative_slope=0.1)
        x = self.lin2(x)
        return x.view(-1)

In [97]:
class Model(nn.Module):
    def __init__(
        self,
        hidden_channels,
        num_heads_GAT,
        dropout_p_GAT,
        edge_dim_GAT,
        momentum_GAT,
        dim_model,
        num_heads_TR,
        num_encoder_layers_TR,
        num_decoder_layers_TR,
        dropout_p_TR,
        n_edges,
        n_nodes,
    ):
        super().__init__()
        self.encoder = GNNEncoder(
            hidden_channels, num_heads_GAT, dropout_p_GAT, edge_dim_GAT, momentum_GAT
        )  # node embedding with GAT
        self.transformer = Transformer(
            dim_model,
            num_heads_TR,
            num_encoder_layers_TR,
            num_decoder_layers_TR,
            dropout_p_TR,
        )
        self.decoder = EdgeDecoder(hidden_channels, num_heads_GAT, n_edges, n_nodes)

    def forward(self, x, edge_indices, edge_attrs):
        src_embedds = []
        for i in range(x.shape[0]):
            src_embedds.append(self.encoder(x[i], edge_indices[i], edge_attrs[i]))
        src_embedds = torch.stack(src_embedds)
        trg_embedds = src_embedds[-1].unsqueeze(0)
        temporal_node_embedds = self.transformer(src_embedds, trg_embedds)
        temporal_node_embedds = temporal_node_embedds.squeeze(0)
        edge_predictions = self.decoder(temporal_node_embedds)
        return edge_predictions

In [98]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model(
    hidden_channels=32,
    num_heads_GAT=2,
    dropout_p_GAT=0.1,
    edge_dim_GAT=1,  # edge attributes
    momentum_GAT=0.1,
    dim_model=32 * 2,  # hidden_channels * num_heads_GAT
    num_heads_TR=2,
    num_encoder_layers_TR=2,
    num_decoder_layers_TR=2,
    dropout_p_TR=0.1,
    n_edges=n_edges,
    n_nodes=n_nodes,
)

c:\Users\lemuelkl\anaconda3\envs\euics\Lib\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [99]:
def train(model, data, window_size, num_epochs, lr):
    model = model.to(device)
    data = [d.to(device) for d in data]
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    for epoch in range(num_epochs):
        # for m in tqdm(range(len(data) - window_size)):
        loss_sum = None
        for m in range(len(data) - window_size):
            optimizer.zero_grad()
            x = torch.stack([data[m + i].x for i in range(window_size)])
            edge_indices = torch.stack(
                [data[m + i].edge_index for i in range(window_size)]
            )
            edge_attrs = torch.stack(
                [data[m + i].edge_attr for i in range(window_size)]
            )
            y = data[m + window_size].y
            y_pred = model(x, edge_indices, edge_attrs)
            loss = criterion(y_pred, y)
            # print(f"Epoch {epoch}, Loss {loss.item()}")
            if loss_sum is None:
                loss_sum = loss
            else:
                loss_sum += loss
            if m % 24 * 7 == 0 or m == len(data) - window_size - 1:
                diff = y.squeeze() - y_pred
                diff = diff.detach().cpu().numpy()
                print(f"Epoch {epoch}, m={m}", diff.mean())
                loss_sum.backward()
                optimizer.step()
                optimizer.zero_grad()
                loss_sum = None

    return model

In [101]:
snapshots = []
for i in range(len(datetime_intersect)):
    x = torch.tensor(node_features[i], dtype=torch.float)
    edge_index = torch.tensor(edge_indices[i], dtype=torch.long)
    edge_attr = torch.tensor(edge_attributes[i], dtype=torch.float)
    y = torch.tensor(edge_labels[i], dtype=torch.float)
    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)
    snapshots.append(data)
print(len(snapshots))
train(model, snapshots, window_size=24 * 4, num_epochs=100, lr=1e-9)

43729


c:\Users\lemuelkl\anaconda3\envs\euics\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 0, m=0 455.17493
Epoch 0, m=24 445.8124
Epoch 0, m=48 409.51834
Epoch 0, m=72 359.78577
Epoch 0, m=96 419.98407
Epoch 0, m=120 446.94714
Epoch 0, m=144 421.63446
Epoch 0, m=168 400.60724
Epoch 0, m=192 403.40442
Epoch 0, m=216 400.99054
Epoch 0, m=240 396.19318
Epoch 0, m=264 392.6469
Epoch 0, m=288 315.49377
Epoch 0, m=312 377.28555
Epoch 0, m=336 389.22055
Epoch 0, m=360 361.6402
Epoch 0, m=384 440.72812
Epoch 0, m=408 348.7458
Epoch 0, m=432 424.44574
Epoch 0, m=456 358.2893
Epoch 0, m=480 350.63287
Epoch 0, m=504 392.09674
Epoch 0, m=528 361.40875
Epoch 0, m=552 427.5383
Epoch 0, m=576 481.68613
Epoch 0, m=600 506.83096
Epoch 0, m=624 510.49695
Epoch 0, m=648 512.7706
Epoch 0, m=672 502.1629
Epoch 0, m=696 416.30396
Epoch 0, m=720 486.72626
Epoch 0, m=744 485.39386
Epoch 0, m=768 502.9057
Epoch 0, m=792 530.1548
Epoch 0, m=816 547.96326
Epoch 0, m=840 524.4026
Epoch 0, m=864 501.1123
Epoch 0, m=888 519.0797
Epoch 0, m=912 494.92566
Epoch 0, m=936 531.9535
Epoch 0, m=960 504.1